In [1]:
import numpy as np
from qiskit.visualization import array_to_latex
from qiskit import QuantumCircuit, transpile, assemble, Aer
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from numpy.random import randint
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import pandas as pd
import qiskit.quantum_info as qi

In [2]:
#Step one is to create an array of bits to send to Bob
def create_alice_bits(n):
    bits = randint(2, size=n)
    return bits

#Then we make another array of the same length to create the basis states for alice to encode the message
#Step one is to create an array of bits to send to Bob
def create_alice_bases(n):
    bases = randint(2, size=n)
    return bases

def create_bob_bases(n):
    bases = randint(2, size=n)
    return bases

def create_eve_bases(n):
    bases = randint(2, size=n)
    return bases

In [10]:
#This function will create a quantum circuit that produces the |00> |01> |10> or |11> configuration
def create_message(n,bits,bases):
    qc_out = []
    if len(bits)==len(bases):
        for i in range(n):
            #create basis vectors
            qc = QuantumCircuit(1,1) #need to update this for noise model
            if bases[i] == 0:
                if bits[i] == 0:
                    pass
                else:
                    qc.x(0)
            else:
                if bits[i] == 0:
                    qc.h(0)
                else:
                    qc.x(0)
                    qc.h(0)
                    
            encode_vec = qi.Statevector.from_instruction(qc)
            qc.barrier()
            qc_out.append(qc)
            qc.draw()
    else:
        print("Bits and basis vectors are unequal")
    return    qc_out

In [26]:
def measure_message(n,bases, qc):
    
    measurements = []
    for i in range(n):
        
        if bases[i] == 0:
            #Measure in z basis
            qc[i].measure(0,0)
        else:
            #Measure in x basis
            qc[i].h(0)
            qc[i].measure(0,0)
            
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(qc[i], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
        
    return measurements

In [27]:
def public_key(n):
      
    #Create bits and bases
    a_bits =  create_alice_bits(n)
    a_bases = create_alice_bases(n)
    b_bases = create_bob_bases(n)
    e_bases = create_eve_bases(n) 
    
    #process to get public key
    alice_message = create_message(n,a_bits,a_bases)
    bobs_message = measure_message(n,b_bases,alice_message)
     
    return bobs_message, a_bits, a_bases, b_bases, e_bases

In [28]:
def intercepted_key(n):
      
    #Create bits and bases
    a_bits =  create_alice_bits(n)
    a_bases = create_alice_bases(n)
    b_bases = create_bob_bases(n)
    e_bases = create_eve_bases(n) 
    
    #process to get public key
    alice_message = create_message(n,a_bits,a_bases)
    #Eve present
    eve_message = measure_message(n,e_bases,alice_message)
    bobs_message = measure_message(n,b_bases,alice_message)
     
    return bobs_message, a_bits, a_bases, b_bases, e_bases

In [29]:
def filter_bits(n, bits,bases1,bases2):

    filtered_bits = []
    #filter all the bits for the message and see if they line up
    for i in range(n):
        if bases1[i] == bases2[i]:
            filtered_bits.append(bits[i])
        else:
            pass
        
    return  filtered_bits

In [30]:
def get_key(n,eve_present):
    
    n=n
    eve_present=eve_present
    
    if eve_present==0:
        key = public_key(n=n)
    else:
        key = intercepted_key(n=n)
        print("oh nooooooooooo, my key has been taken")

    b_mess  = key[0]
    a_bits  = key[1]
    a_bases = key[2]
    b_bases = key[3]
    e_bases = key[4]

    print("bobs measurement is" ,b_mess)
    print("alices bits are    " ,a_bits)
    print("alices bases are   " ,a_bases)
    print("bobs bases are     " ,b_bases)
    print("eves bases are     " ,e_bases)

    alices_bits = filter_bits(n,a_bits,a_bases,b_bases)
    bobs_bits   = filter_bits(n,b_mess,a_bases,b_bases)

    print("alices key is      ", alices_bits)
    print("bobs key is        ", bobs_bits)
    matched_key = []
    
    for i in range(len(alices_bits)):
        if alices_bits[i]==bobs_bits[i]:
            matched_key.append(alices_bits[i])
    print("matched key is", matched_key)
    
    percent_of_matched_bits = len(matched_key)/len(alices_bits)*100
    print("percent of matched bits is =",percent_of_matched_bits)

In [32]:
get_key(int(input()), int(input()))

10
1
oh nooooooooooo, my key has been taken
bobs measurement is [1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
alices bits are     [1 1 1 0 1 1 1 1 1 0]
alices bases are    [1 0 0 0 0 1 1 1 1 1]
bobs bases are      [0 1 1 0 1 1 1 1 0 1]
eves bases are      [1 1 0 1 1 0 1 0 0 0]
alices key is       [0, 1, 1, 1, 0]
bobs key is         [0, 1, 1, 1, 1]
matched key is [0, 1, 1, 1]
percent of matched bits is = 80.0
